In [2]:
# 1. Скрипт создания ВБД (create_db.py)
# Этот скрипт берет medical_dictionary.pdf, разбивает его на части (чанки) и сохраняет их в FAISS, 
# используя локальную модель эмбеддинга.

import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
PDF_SOURCE = "medical_dictionary.pdf"  # Имя вашего файла
DB_DIR = "medical_vector_store"        # Папка, куда сохранится база
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"

def create_vector_db():
    if not os.path.exists(PDF_SOURCE):
        print(f"ОШИБКА: Файл {PDF_SOURCE} не найден.")
        return

    print(f"1. Загрузка PDF: {PDF_SOURCE}...")
    loader = PyPDFLoader(PDF_SOURCE)
    documents = loader.load()
    print(f"   Загружено страниц: {len(documents)}")

    # 2. Разбивка на чанки (оптимизация)
    # chunk_size=500 и overlap=200 — разбивает весь документ на части с перехлестом.
    # Это позволяет захватить определение целиком и сохранить контекст на границах.
    print("2. Разбивка текста на чанки...")
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=200,
        separators=["\n\n", "\n", " ", ""] # Стараемся делить по абзацам
    )
    docs = text_splitter.split_documents(documents)
    print(f"   Создано фрагментов: {len(docs)}")

    # 3. Инициализация эмбеддингов (ровно как в вашем скрипте)
    print(f"3. Загрузка модели из {LOCAL_MODEL_DIR}...")
    embeddings = HuggingFaceEmbeddings(
        model_name=LOCAL_MODEL_DIR,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        show_progress=True
    )

    # 4. Создание и сохранение FAISS индекса
    print("4. Создание векторного индекса FAISS...")
    vectorstore = FAISS.from_documents(docs, embeddings)
    
    vectorstore.save_local(DB_DIR)
    print(f"ГОТОВО! База сохранена в папку: {DB_DIR}")

if __name__ == "__main__":
    create_vector_db()

1. Загрузка PDF: medical_dictionary.pdf...
   Загружено страниц: 483
2. Разбивка текста на чанки...
   Создано фрагментов: 6887
3. Загрузка модели из /Users/ivannemcenko/Models/all-MiniLM-L6-v2...


/var/folders/wc/x6r7l4gd2gd0sy7wrqxrpjyr0000gn/T/ipykernel_72523/608091464.py:40: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /Users/ivannemcenko/Models/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


4. Создание векторного индекса FAISS...


Batches:   0%|          | 0/216 [00:00<?, ?it/s]

ГОТОВО! База сохранена в папку: medical_vector_store


In [2]:
# Этот скрипт реализует логику ретривера (-R-ag)
# Загружаем модель эмбеддинга через HuggingFaceEmbeddings
# функция get_doctor_context реализует логику семантического поиска по ВБД
# поиск происходит с помощью метода similarity_search_with_relevance_scores,
# то есть вынимаются схожие по смыслу документы вместе с Score (оценкой схожести).
# Затем автомат сортируются по Score, функция отдает топ-3 документа дальше для аугментации и генерации в LLM (r-AG-)

import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
DB_DIR = "medical_vector_store"
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"
TOP_K = 3

# Один раз загружаю модель эмбеддинга и Векторную Базу Данных
print(f"Загрузка модели эмбеддингов из: {LOCAL_MODEL_DIR}...")
embeddings = HuggingFaceEmbeddings(
        model_name=LOCAL_MODEL_DIR,
        model_kwargs={"device": "cpu"},
        encode_kwargs={"normalize_embeddings": True},
        show_progress=False
    )

print(f"Загрузка векторной базы из: {DB_DIR}...")
try:
        vectorstore = FAISS.load_local(
            DB_DIR,
            embeddings,
            allow_dangerous_deserialization=True,
        )
except Exception as e:
        print(f"Ошибка загрузки базы: {e}")


def get_doctor_context(query_text):
    # Поиск документов
    results = vectorstore.similarity_search_with_relevance_scores(
        query_text, 
        k=TOP_K
    )

    if not results:
        print("Результаты не найдены.")
        return ""

    # === ФОРМИРОВАНИЕ DOCTOR_DOC ===
    # Мы создаем список, куда сложим все найденные ответы
    doc_entries = []
    
    for i, (doc, score) in enumerate(results, 1):
        # Основной текст находится в атрибуте page_content
        pdf_content = doc.page_content
        
        # Также можем вытащить номер страницы, если он есть (PyPDFLoader его создает)
        page_num = doc.metadata.get('page', 'N/A')
        
        # Форматируем блок.
        entry = (f"--- RETRIEVED FROM DICTIONARY #{i} (Page: {page_num}, Score: {score:.2f}) ---\n"
                 f"CONTENT:\n{pdf_content}")
        
        doc_entries.append(entry)

    # 4. Склеиваем всё в одну строку
    # Добавляем заголовок раздела и объединяем кейсы через отступы
    header = "\n\n### REFERENCE DATA FROM MEDICAL DICTIONARY:\n"
    DOCTOR_DOC = header + "\n\n".join(doc_entries)

    return DOCTOR_DOC

Загрузка модели эмбеддингов из: /Users/ivannemcenko/Models/all-MiniLM-L6-v2...


/var/folders/wc/x6r7l4gd2gd0sy7wrqxrpjyr0000gn/T/ipykernel_79537/4101899894.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: /Users/ivannemcenko/Models/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Загрузка векторной базы из: medical_vector_store...


In [3]:
# Реализована логика работы системы семантического поиска для проверки работоспособности системы

import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# === КОНФИГУРАЦИЯ ===
DB_DIR = "medical_vector_store"
LOCAL_MODEL_DIR = "/Users/ivannemcenko/Models/all-MiniLM-L6-v2"
TEST_QUERY = "What is obsessive-compulsive personality disorder?"

DOCTOR_DOC = get_doctor_context(TEST_QUERY)
print(DOCTOR_DOC)



### REFERENCE DATA FROM MEDICAL DICTIONARY:
--- RETRIEVED FROM DICTIONARY #1 (Page: 315, Score: 0.68) ---
CONTENT:
observer experiences when observing the same
material more than once.
obsessive-compulsive disorder An anxiety dis-
order that is characterized by obsessive thoughts and
compulsive actions. Abbreviated OCD. The obsessive
thoughts are unwanted ideas or impulses that
repeatedly well up in the mind of the person with
OCD. These thoughts are intrusive and unpleasant,
and they produce a high degree of anxiety. In
response to their obsessions, most people with OCD

--- RETRIEVED FROM DICTIONARY #2 (Page: 315, Score: 0.65) ---
CONTENT:
pulsive personality disorder, which is a personality
disorder rather than an anxiety disorder.
obsessive-compulsive personality disorder
A personality disorder that is characterized by per-
vasive preoccupation with orderliness, perfection-
ism, and interpersonal control. Abbreviated OCPD.
OCPD may feature a preoccupation with details,
rules, lis

**Все работает корректно, документы релевантные!**

In [3]:
# Данный скрипт реализует логику работы этап Аугментации (rAg) документов (из семантического поиска) в контекст LLM через messages
# Так же в этом скрипте реализуется этап Генерации (raG) ответа от LLM на основе найденных документов

import json
import time
import io
import re
from contextlib import redirect_stdout
from mlx_lm import load, generate

# --- CONFIG ---
MODEL_PATH = "./Base_Line_Qwen3B_MLX_INT8"
SYSTEM_PROMPT = """
You are an experienced medical doctor providing telemedicine consultation.

CRITICAL GUIDELINES:
1. Base your response on the provided similar medical cases
2. Be professional, empathetic, and evidence-based
3. Provide clear, actionable advice
4. Include warning signs for emergency situations
5. Always recommend consulting with a healthcare provider
6. Never diagnose definitively without examination

RESPONSE STRUCTURE:
- Acknowledge the patient's concern
- Reference similar cases if relevant
- Provide practical guidance
- List red flags requiring immediate attention
- Suggest next steps

DISCLAIMER: This is informational only, not medical advice.
"""
TEST_QUERY = "What is obsessive-compulsive personality disorder?"
MAX_TOKENS = 1024

# --- LOAD MODEL ---
model, tokenizer = load(MODEL_PATH)


# RAG - логика
DOCTOR_DOC = get_doctor_context(TEST_QUERY)
RAG_content = SYSTEM_PROMPT + DOCTOR_DOC

messages = [
    {"role": "system", "content": RAG_content},
    {"role": "user", "content": TEST_QUERY},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

stdout_buffer = io.StringIO()
start_time = time.time()

with redirect_stdout(stdout_buffer):
    response = generate(
            model=model,
            tokenizer=tokenizer,
            prompt=prompt,
            max_tokens=MAX_TOKENS,
            verbose=True # verbose нужен для парсинга токенов (чтобы логировать расход токенов)
        )

latency_sec = round(time.time() - start_time, 4)
stdout_text = stdout_buffer.getvalue()

# --- PARSE TOKENS ---
prompt_tokens = 0
generated_tokens = 0
    
p_match = re.search(r"Prompt:\s+(\d+)", stdout_text)
g_match = re.search(r"Generation:\s+(\d+)", stdout_text)
    
if p_match: prompt_tokens = int(p_match.group(1))
if g_match: generated_tokens = int(g_match.group(1))
total_tokens = prompt_tokens + generated_tokens

print(f"Тестовый запрос: {TEST_QUERY}\n")
print(f"Найденные документы ретривером: {DOCTOR_DOC}\n\n")
print(f"Контекст, который ушел в LLM: {messages}\n\n")
print(f"Ответ LLM:  {response.strip()}\n\n")
print(f"latency_sec: {latency_sec}")
print(f"prompt_tokens: {prompt_tokens}")
print(f"generated_tokens: {generated_tokens}")
print(f"total_tokens: {total_tokens}")

Тестовый запрос: What is obsessive-compulsive personality disorder?

Найденные документы ретривером: 

### REFERENCE DATA FROM MEDICAL DICTIONARY:
--- RETRIEVED FROM DICTIONARY #1 (Page: 315, Score: 0.68) ---
CONTENT:
observer experiences when observing the same
material more than once.
obsessive-compulsive disorder An anxiety dis-
order that is characterized by obsessive thoughts and
compulsive actions. Abbreviated OCD. The obsessive
thoughts are unwanted ideas or impulses that
repeatedly well up in the mind of the person with
OCD. These thoughts are intrusive and unpleasant,
and they produce a high degree of anxiety. In
response to their obsessions, most people with OCD

--- RETRIEVED FROM DICTIONARY #2 (Page: 315, Score: 0.65) ---
CONTENT:
pulsive personality disorder, which is a personality
disorder rather than an anxiety disorder.
obsessive-compulsive personality disorder
A personality disorder that is characterized by per-
vasive preoccupation with orderliness, perfection-
ism, a

**Этапы аугментации и генерации так же работают отлично!**
**RAG готов к работе!**